<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">Introduction</h1>
<p>
    Kleinian functions, also known as the generalised Wierstrass $\wp_{ij}$ functions, are important tools for studying hyperelliptic curves. The main objective of this notebook is to present code that can compute Kleinian functions for a hyperelliptic curve of genus 2.  While the primary focus is on the computation aspect, related concepts and background information are provided for context. 
</p>
<p>
This notebook is based on the work of Julia Bernatska <a href="https://arxiv.org/abs/2407.05632" target="_blank">[1]</a> and the code attached to an article posted on WolframCommunity <a href="https://community.wolfram.com/groups/-/m/t/3243472" target="_blank">[2]</a>. It also draws from the SageMath Riemann surfaces documentation <em>SageMath Riemann surfaces</em> documentation <a href="https://doc.sagemath.org/html/en/reference/curves/sage/schemes/riemann_surfaces/riemann_surface.html#sage.schemes.riemann_surfaces.riemann_surface.RiemannSurface"
target="_blank">[3]</a>. All additional references are listed in the Literature section at the end of the notebook.
</p>
</div>


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">1. Preliminaries</h1>
<p>
    Many works on hyperelliptic functions and Riemann surfaces use similar notation, which can lead to potential naming collisions. To avoid such collisions, a legend of notations used in this notebook is provided below.
</p>
<h3 style="font-size: 24px;">1.1. Legend</h3>
    <ul>
        <li>$\mathscr{C}$ - Curve</li>
        <li>$X$ - Riemann surface</li>
        <li>$e_i$ - branch points</li>
        <li>$w$ - an Abelian differential on the Riemann surface $X$</li>
        <li>$W$ - general Abelian integral</li>
        <li>$A$, $B$ - A- and B-periods of the differential $w=dW$</li>        
        <li>$du$ - First kind differential</li>
        <li>$dr$ - Second kind differential</li>
        <li>$\mathfrak{a}_i, \mathfrak{b}_i$ - Canonical homology cycles</li>
        <li>$\omega$ -  First kind integrals</li>
        <li>$\eta$ - Second kind integrals</li>
        <li>$\mathcal{A}$ - Abel map</li>
        <li>$\mathfrak{m}(\mathbf{u})$ - A meromorphic function of $\mathbf{u}\in\mathbb{C}^g$</li>
        <li>$\Omega$ - Period matrices </li>
        <li>$\Lambda$ - Lattice</li>
        <li>$\mathrm{Jac}(\mathscr{C})$ - Jacobian variety of the curve $\mathscr{C}$</li>
        <li>$D$ - Divisor  on the Riemann surface $X$</li>
</ul>
  
</div>

<div>
<h3 style="font-family: 'Latin Modern Roman', Times, serif; font-size: 24px;"> 1.2. Hyperelliptic curve</h3>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    In studies on hyperelliptic functions, various conventions for defining a curve, and consequently the hyperelliptic functions, can be found. In this notebook, we adopt the convention presented in <a href="https://arxiv.org/abs/2407.05632" target="_blank">[1]</a>. <br>
    <br>
    Additionally, conventions for curves in both general and canonical forms are also discussed in dedicated notebooks: `canonical-name.ipynb` and `general-name.ipynb`. Please refer to these notebooks for further details.
</p>
</div>
 


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 1.</h4>
    <p>    
        The hyperelliptic curve is defined by 
        $$
            \mathscr{C} = \{ (x,y)\in \mathbb{C}^2 \mid f(x,y)=0 \},
        $$
        where
        $$
            f(x,y) = -y^2 + x^{2g+1} + \sum_{i=0}^{2g}\lambda_{2i+2}x^{2g-i}, \quad \lambda_{k\leq0}=0,\; \lambda_k \in \mathbb{R}.$
        $$
    </p>
</div>


<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 1.</h4>
    <p>
    <ul>
        We assume $\lambda_k \in \mathbb{R}$, instead of $\lambda_k \in \mathbb{C}$ which can be found in various texts, because we would like to apply these functions to physical equations first.
    </ul>            
  </p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    So for a curve of genus $g=2$ we get
    $$
        f(x,y) = -y^2 + x^5 + \lambda_2 x^4 + \lambda_4 x^3+ \lambda_6 x^2+ \lambda_8 x+ \lambda_{10},
    $$   
    Let's load the appropriate SageMath library and define the curve $\mathscr{C}$
</p>    
</div>

In [3]:
from sage.schemes.riemann_surfaces.riemann_surface import RiemannSurface

In [4]:
# Defines the lambda coefficients
lambda2 = 12.0
lambda4 = 2.0
lambda6  = 0.9
lambda8  = 21.0
lambda10  = 10.0

<div style="background-color:  rgba(255, 153, 153, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(255, 51,51); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(255, 51,51);">Important 1.</h4>
    <p>
        The coefficients above must be real-floating-point numbers        
    </p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Since the current Sage library only works well on the field of rational numbers, we have to approximate all function coefficients by these numbers.
</p>
</div>

In [5]:
# Rational approximation
l2 = lambda2.nearby_rational(max_error=1e-10)
l4 = lambda4.nearby_rational(max_error=1e-10)
l6 = lambda6.nearby_rational(max_error=1e-10)
l8 = lambda8.nearby_rational(max_error=1e-10)
l10 = lambda10.nearby_rational(max_error=1e-10)

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Next, we need to define the variables $x$ and $y$ in the ring of polynomials over the rational numbers.
</p>
</div>

In [6]:
R.<x, y> = PolynomialRing(QQ, 2)

<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 2.</h4>
    <p>
        The order of variables can be important depending on how you write your polynomial. Make sure you use the same order in your polynomial expression.        
    </p>
</div>

In [7]:
# Defining the polynomial f
f = -y^2 + x^5 + l2*x^4 + l4*x^3 + l6*x^2 + l8*x + l10

<div>
<h3 style="font-family: 'Latin Modern Roman', Times, serif; font-size: 24px;">1.3. Riemann surface</h3>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 2. </h4>
    <p>    
        A Riemann surface $X$ is a connected two-dimensional topological manifold with a complex-analytic structure on it. 
    </p>    
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    The SageMath RiemannSurface library provides a function to generate the appropriate Riemann surface, on which we will continue our work.
</p>
</div>

In [8]:
S = RiemannSurface(f, prec=100)

<div>
<h3 style="font-family: 'Latin Modern Roman', Times, serif; font-size: 24px;">1.4. Branch points</h3>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    We define a function to compute and display all branch points $e_i$ based on the given coefficients $\lambda_{2i}$. Branch points correspond to the zeros of the polynomial defined by $f(x, y) = 0$. Specifically, the polynomial is given by:
$$
    y^2 = (x - e_1)(x - e_2)(x - e_3)(x - e_4)(x - e_5).
$$
Here, the branch points $e_i$ are the roots of the polynomial on the right-hand side, which describe the structure of the Riemann surface associated with $f(x, y) = 0$.
</p>
</div>

In [9]:
def find_branch_points(ll2, ll4, ll6, ll8, ll10):
    # We define a ring of polynomials over the field of complex numbers
    CC_poly.<x> = PolynomialRing(CC)
    
    # We create a polynomial
    pol = x^5 + ll2*x^4 + ll4*x^3 + ll6*x^2 + ll8*x + ll10
    
    # We find roots
    roots = pol.roots(multiplicities=False)
    
    # We add a point at infinity if the degree of the polynomial is odd
    if pol.degree() % 2 == 1:
        roots.append(infinity)
    
    return roots

<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 3.</h4>
    <p>
        The <code style="color: rgb(81,206,9);font-size: 13px;">multiplicities=False</code> parameter in the <code style="color: rgb(81,206,9);font-size: 13px;">roots()</code>  method in Sage has the following meaning:
        <ul>
            <li>When <code style="color: rgb(81,206,9);font-size: 13px;">multiplicities=False</code>  (default): The method returns only the roots of the polynomial, without information about their multiplicities. The result is a list of unique root values.</li>
            <li>When <code style="color: rgb(81,206,9);font-size: 13px;">multiplicities=True</code>: The method returns pairs (root, multiplicity) for each root. The result is a list of tuples, where each tuple contains the root and its multiplicity.</li>
        </ul>            
    </p>
</div>



In [10]:
# Example of use:

branch_points = find_branch_points(l2, l4, l6, l8, l10)
print("Branch points:", *branch_points, sep='\n')

Branch points:
-11.8251161409669
-1.05264146874943
-0.512317960119317
0.695037784917798 - 1.04164546422316*I
0.695037784917798 + 1.04164546422316*I
+Infinity


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">2. First and Second Kind Periods</h1>
<p>
<p>
    To construct periodic functions, we first define a canonical basis for the space of holomorphic differentials, \( \{du_i \mid i = 1, \ldots, g\} \), and the associated meromorphic differentials, \( \{dr_i \mid i = 1, \ldots, g\} \), on the Riemann surface as follows:
    $$
        du_{2i-1} := \frac{x^{g-i} dx}{\partial_y f(x,y)},
    $$
    $$
        dr_{2i-1} := \frac{\mathcal{R}_{2i-1}(x) dx}{\partial_y f(x,y)}, 
    $$
    where
    $$
    \mathcal{R}_{2i-1}(x)=\sum_{k=1}^{2i-1}k\lambda_{4i-2k-2}x^{g-i+k}.
    $$
    For \( g = 2 \), these expressions can be written in vector form as:
    $$
        du= \begin{pmatrix} 
                x\\
                1
            \end{pmatrix} \frac{dx}{-2\sqrt{f(x,y)}},
    $$
    $$
        dr= \begin{pmatrix} 
                \mathcal{R}_{1}(x)\\
                \mathcal{R}_{3}(x)
            \end{pmatrix} \frac{dx}{-2\sqrt{f(x,y)}} = 
                \begin{pmatrix} 
                    x^2\\
                    3x^3 + 2\lambda_2 x^2 + \lambda_4 x
                \end{pmatrix} \frac{dx}{-2\sqrt{f(x,y)}},
    $$
</p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    The holomorphic basis defined above can be compared to the output of the <code style="color: rgb(81,206,9);font-size: 13px;">cohomology_basis()</code> function. In SageMath, <code style="color: rgb(81,206,9);font-size: 13px;">S.cohomology_basis()</code> generates a list of holomorphic differentials, typically represented as polynomials $g(x)$ corresponding to the differentials:
    $$
        \omega = g(x,y) \frac{\, dx}{\partial f / \partial y}
    $$
    where  $f(x, y) = 0$  defines the curve.
</p>
</div>    

In [11]:
S.cohomology_basis()

[1, x]

<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 4.</h4>
    <p>
    The order of elements clearly differs from the convention adopted in <a href="https://arxiv.org/abs/2407.05632" target="_blank">[1]</a>. In this notebook, we adopt Julia Bernatska’s convention, as it provides a consistent framework for our computations.
    </p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">2.1. First  Kind Periods</h2>
</div>

In [12]:
# holomorphic differentials base
holbais=[x,x^0]

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    To compute the period matrices of the first kind, we evaluate the following integrals along the canonical homology cycles $\{\mathfrak{a}_i, \mathfrak{b}_i\}_{i=1}^g$:
    $$
        \omega = (\omega_{ij})= \left( \int_{\mathfrak{a}_j}du_i \right), \quad \omega' = (\omega'_{ij})= \left( \int_{\mathfrak{b}_j}du_i \right). 
    $$
Here, $\omega$ and $\omega'$ are the period matrices corresponding to the $\mathfrak{a}$ and $\mathfrak{b}$ cycles, respectively.

The SageMath function <code style="color: rgb(81,206,9);font-size: 13px;">matrix_of_integral_values(differentials, integration_method='heuristic')</code> can be used to compute the path integrals of the given differentials along the homology basis. The result is a matrix, where each row corresponds to a differential.

If the Riemann surface is given by the equation $f(x,y)=0$, the differentials are encoded by:
    $$
            g(x,y)\frac{dx}{(df/dy)}.
    $$
    <code style="color:  rgb(229,201,18);font-size: 13px;">Input:</code>
    <ul>
        <li><code style="color:  rgb(81,206,9);font-size: 13px;">differentials</code> – a list of polynomials.</li>
        <li><code style="color:  rgb(81,206,9);font-size: 13px;">integration_method</code> – (default: 'heuristic'). String specifying the integration method to use. The options are 'heuristic' and 'rigorous'.</li>
    </ul>
    <code style="color:  rgb(229,201,18);font-size: 13px;">Output:</code>
    A matrix, one row per differential, containing the values of the path integrals along the homology basis of the Riemann surface.
</p>
</div>

In [13]:
MofInt1=S.matrix_of_integral_values(holbais)
# Let's display the matrix in a shortened form so that it will be easy to see its structure
print(MofInt1.n(digits=5))

[     0.13760 - 0.66033*I  -4.9304e-32 - 0.91722*I  -3.9443e-31 + 0.40344*I       1.1746 + 0.25689*I]
[    -0.62448 + 0.28220*I -5.5467e-32 + 0.038862*I  -4.9304e-32 - 0.52554*I      0.14607 + 0.24334*I]


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    The structure of the returned matrix
    $$
    MofInt1 = 
        \begin{pmatrix}
            \omega_{1,1} & \omega_{1,3} & \omega'_{1,1} & \omega'_{1,3} \\
            \omega_{3,1} & \omega_{3,3} & \omega'_{3,1} & \omega'_{3,3}
        \end{pmatrix}
        =
        \begin{array}{|c|c|c|c|c|}
            \hline
            & \mathfrak{a}_1 & \mathfrak{a}_3 & \mathfrak{b}_1 & \mathfrak{b}_3 \\
            \hline
            x & \omega_{1,1} = \int_{\mathfrak{a}_1} du_1 & \omega_{1,3} = \int_{\mathfrak{a}_3} du_1 & \omega'_{1,1} = \int_{\mathfrak{b}_1} du_1 & \omega'_{1,3} = \int_{\mathfrak{b}_3} du_1 \\
            \hline
            1 & \omega_{3,1} = \int_{\mathfrak{a}_1} du_3 & \omega_{3,3} = \int_{\mathfrak{a}_3} du_3 & \omega'_{3,1} = \int_{\mathfrak{b}_1} du_3 & \omega'_{3,3} = \int_{\mathfrak{b}_3} du_3 \\
            \hline
        \end{array}
    $$
</p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
    <p>
    We can compare this with the results of the built-in Sage function: <code style="color:  rgb(81,206,9);font-size: 13px;">period_matrix()</code>, which, for the adopted notational convention, will return a period matrix in the form
    $$
        pM= \begin{pmatrix}
                \omega_{3,1} & \omega_{3,3} & \omega'_{3,1} & \omega'_{3,3} \\
                \omega_{1,1} & \omega_{1,3} & \omega'_{1,1} & \omega'_{3,1}
        \end{pmatrix}  
    $$   
    </p>
</div>

In [14]:
pM=S.period_matrix()
print(pM.n(digits=5))

[   -0.62448 + 0.28220*I 4.3141e-32 + 0.038862*I -4.9304e-32 - 0.52554*I     0.14607 + 0.24334*I]
[    0.13760 - 0.66033*I  2.2187e-31 - 0.91722*I -5.9165e-31 + 0.40344*I      1.1746 + 0.25689*I]


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Before going any further, lat's define a function that will display the matrices in a rounded form so that we can compare them more easily.
</p>
</div>    

In [15]:
def format_complex(z, digits=5, threshold=1e-10):
    real = float(z.real())
    imag = float(z.imag())
    
    # We round very small values to zero
    if abs(real) < threshold:
        real = 0
        if abs(imag) < threshold:
            return "0"
        # We format the result  
        return f"{imag:.{digits}f}*I"
    
    if abs(imag) < threshold:
        # We format the result
        return f"{real:.{digits}f}"

    sign = "+" if imag > 0 else "-"
    return f"{real:.{digits}f} {sign} {abs(imag):.{digits}f}*I"


def ApproxM(matrix, digits=5, threshold=1e-10):
    rows, cols = matrix.nrows(), matrix.ncols()
    
    for i in range(rows):
        formatted_row = [format_complex(matrix[i,j], digits, threshold) for j in range(cols)]
        print("\t".join(formatted_row))

In [16]:
ApproxM(pM)

-0.62448 + 0.28220*I	0.03886*I	-0.52554*I	0.14607 + 0.24334*I
0.13760 - 0.66033*I	-0.91722*I	0.40344*I	1.17459 + 0.25689*I


In [17]:
ApproxM(MofInt1)

0.13760 - 0.66033*I	-0.91722*I	0.40344*I	1.17459 + 0.25689*I
-0.62448 + 0.28220*I	0.03886*I	-0.52554*I	0.14607 + 0.24334*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
    <p>
    In what follows we use the function <code style="color:  rgb(81,206,9);font-size: 13px;">matrix_of_integral_values()</code> instead of <code style="color:  rgb(81,206,9);font-size: 13px;">period_matrix()</code> because it allows us to calculate periodic matrices of the second kind.
    </p>
</div>

In [18]:
# Extract the omega-periods (first two columns)
omega = MofInt1[:, 0:2]

# Extract the omega'-periods (last two columns)
omegaP = MofInt1[:, 2:4]

ApproxM(omega)
print()
ApproxM(omegaP)

0.13760 - 0.66033*I	-0.91722*I
-0.62448 + 0.28220*I	0.03886*I

0.40344*I	1.17459 + 0.25689*I
-0.52554*I	0.14607 + 0.24334*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
    <p>
    Now we calculate the matrix
    $$
        \tau=\omega^{-1}\omega'
    $$
    which belongs to the Siegel upper half-space. Hence it should satisfy two conditions:
    <ul>
        <li>Symmetry: $$ \tau^T = \tau$$ 
        <li>Positive definiteness of the imaginary part $$ Im(\tau)>0$$
    </ul>
    </p>
</div>

In [19]:
tau= omega.inverse() * omegaP

# Displaying the result
print(tau.n(digits=5))

[-0.28894 + 0.70313*I -0.12636 - 0.46286*I]
[-0.12636 - 0.46286*I  -0.25854 + 1.6328*I]


In [20]:
# Test of the symmetry
print(tau-tau.transpose().n(digits=5))

[0.00000 0.00000]
[0.00000 0.00000]


In [21]:
# Test of positivity
# Calculating the complex part of the tau matrix
tauImag = tau.apply_map(lambda x: x.imag())

# Calculate the eigenvalues
eigenvalues = tauImag.eigenvalues()

# Checking if all eigenvalues are positive
all_positive = all(e > 0 for e in eigenvalues)

# Displaying the result
eigenvalues, all_positive

([1.8239307353738361091933820951, 0.51198440541283078719843599741], True)

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">2.2. Second Kind Periods</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
To calculate the second kind period matrices, we need to calculate the following integrals along the canonical homology cycles $\{ \mathfrak{a}_i, \mathfrak{b}_i\}_{i=1}^g$
    <div style="text-align: center;">
        $\eta = (\eta_{ij})= \left( \int_{\mathfrak{a}_j}dr_i \right), \quad \eta' = (\eta'_{ij})= \left( \int_{\mathfrak{b}_j}dr_i \right). $
    </div>
</p>
</div>

In [22]:
# meromorphic differentials base
merbais=[x^2, 3*x^3 + 2*l2*x^2 + l4*x]

In [23]:
MofInt2=S.matrix_of_integral_values(merbais)
# Let's display the matrix in a shortened form so that it will be easy to see its structure
ApproxM(MofInt2)

0.15574 + 0.20627*I	0.08372*I	-0.32882*I	-7.07532 + 0.12255*I
4.74867 + 3.09497*I	-0.04832*I	-6.23827*I	-2.96268 + 3.14329*I


In [24]:
# Extract the omega-periods (first two columns)
eta = MofInt2[:, 0:2]

# Extract the omega'-periods (last two columns)
etaP = MofInt2[:, 2:4]

ApproxM(eta)
print()
ApproxM(etaP)

0.15574 + 0.20627*I	0.08372*I
4.74867 + 3.09497*I	-0.04832*I

-0.32882*I	-7.07532 + 0.12255*I
-6.23827*I	-2.96268 + 3.14329*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
    <p>
    We can compute $\kappa$, given by
    $$
        \kappa=\eta\; \omega^{-1}
    $$
    </p>
</div>

In [25]:
omega_inv=Matrix(omega).inverse()
kappa = eta*omega_inv
ApproxM(kappa)

-0.09763 - 0.01261*I	-0.14978 - 0.29754*I
-0.14978 - 0.29754*I	-4.77835 - 7.02263*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">3. Legendre relation</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Next, we perform another test. The unnormalised period matrices of the first kind, $\omega$ and $\omega'$, and the second kind, $\eta$ and $\eta'$, should satisfy the Legendre relation:
    $$
        \Omega^T J \Omega = 2\pi i J
    $$
    where
    $$
        \Omega=
            \begin{pmatrix}
                \omega && \omega'\\
                \eta && \eta'\\
            \end{pmatrix}, \quad 
        J=        
            \begin{pmatrix}
                0 && -1_g\\
                1_g && 0\\
            \end{pmatrix}
    $$
</p>
</div>

In [26]:
# Omega matrix
Omega = block_matrix([
    [omega, omegaP],
    [eta, etaP]
])

# Converting lists to matrices
zeroM = Matrix([[0.0, 0.0], [0.0, 0.0]])
mOneg = Matrix([[-1.0, 0.0], [0.0, -1.0]])
Oneg = Matrix([[1.0, 0.0], [0.0, 1.0]])

# J matrix
J = block_matrix([
    [zeroM, mOneg],
    [Oneg, zeroM]
])    


print("Omega Matrix:")
print(Omega.n(digits=5))
print()
print("J Matrix:")
print(J.n(digits=5))

Omega Matrix:
[     0.13760 - 0.66033*I  -4.9304e-32 - 0.91722*I| -3.9443e-31 + 0.40344*I       1.1746 + 0.25689*I]
[    -0.62448 + 0.28220*I -5.5467e-32 + 0.038862*I| -4.9304e-32 - 0.52554*I      0.14607 + 0.24334*I]
[-------------------------------------------------+-------------------------------------------------]
[     0.15574 + 0.20627*I  7.8886e-31 + 0.083724*I| -8.3816e-31 - 0.32882*I      -7.0753 + 0.12255*I]
[       4.7487 + 3.0950*I  1.1360e-28 - 0.048319*I|   2.2088e-29 - 6.2383*I       -2.9627 + 3.1433*I]

J Matrix:
[0.00000 0.00000|-1.0000 0.00000]
[0.00000 0.00000|0.00000 -1.0000]
[---------------+---------------]
[ 1.0000 0.00000|0.00000 0.00000]
[0.00000  1.0000|0.00000 0.00000]


In [27]:
import numpy as np

pi = np.pi
left=Omega.transpose()*J*Omega
right = 2*pi*I*J
result = left - right
ApproxM(result)

0	0	0	0
0	0	0	0
0	0	0	0
0	0	0	0


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">4. Characteristics of branch points</h1>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 3.</h4>
    <p>   
    A characteristic is a $2\times g$ matrix $[\varepsilon] = (\mathbf{\varepsilon}', \mathbf{\varepsilon})^T$ with real values within the interval $[0,2)$.
    Every point $u$ in the funcamental domain $\mathrm{Jac}(\mathscr{C})$ can be represented by its characteristic $[\varepsilon]$:
        $$
            u = \frac{1}{2} \omega \mathbf{\varepsilon} + \frac{1}{2}\omega' \mathbf{\varepsilon}'.
        $$
    In the hyperelliptic case, the Abel images of branch points and any combination of branch points are described by characteristics with 1 or 0, which are called <i>half-integer</i> characteristics.
</p>
</div>   

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    The half-integer characteristics are odd whenever 
    $$
        \varepsilon^T \varepsilon'=0 \pmod{2},
    $$
    and even when 
    $$
        \varepsilon^T \varepsilon'= 1\pmod{2},
    $$
    For an Abelian image of a branch points
    $$
        \mathcal{A}_{i,0} = \int^{\infty}_{e_i} du
    $$
    we can write
    $$
        \mathcal{A}_{i,0} = \frac{1}{2} \omega \mathbf{\varepsilon}_i + \frac{1}{2}\omega' \mathbf{\varepsilon}'_i.
    $$
    Hence the characteristics of $\mathcal{A}_{i,0}$ is given by
    $$
        [\mathcal{A}_{i,0}] = 
            \begin{pmatrix}
                \mathbf{\varepsilon'}_i^T\\
                \mathbf{\varepsilon}_i^T
            \end{pmatrix} =
            \begin{pmatrix}
                \varepsilon'_{i,1} & \ldots & \varepsilon'_{i,g} \\
                \varepsilon_{i,1} & \ldots &\varepsilon_{i,g}
            \end{pmatrix}.
    $$
    For $g=2$ 
    $$
        [\mathcal{A}_{i,0}] = 
            \begin{pmatrix}
                \varepsilon'_{i,1} & \varepsilon'_{i,2} \\
                \varepsilon_{i,1} & \varepsilon_{i,2}
            \end{pmatrix}
    $$
</p>
</div>

In [28]:
# We define the genus variable
genus = S.genus  


eChars = [[[0 for k in range(genus)], [1 for k in range(genus)]], 
          [[0 for k in range(genus)] for i in range(2)]]

# Adaptation of a Do loop from Julia Bernatska’s Mathematica notebook
for l in range(genus):
    # let's note the indexing, which must be adjusted by
    eChars.insert(0, 
        [[(eChars[0][0][k] + kronecker_delta(k+1, genus - l) + 
           kronecker_delta(k+1, genus - l + 1)) % 2 for k in range(genus)],
         [(eChars[0][1][k] + 0) % 2 for k in range(genus)]]
    )

    eChars.insert(0,
        [[(eChars[0][0][k] + 0) % 2 for k in range(genus)],
         [(eChars[0][1][k] + kronecker_delta(k+1, genus - l)) % 2 for k in range(genus)]]
    )

# We display matrices
seen_matrices = []
for i in range(len(eChars)):
    current_matrix = matrix(eChars[i])
    if current_matrix not in seen_matrices:
        seen_matrices.append(current_matrix)
        print(current_matrix)
        print()

[1 0]
[0 0]

[1 0]
[1 0]

[0 1]
[1 0]

[0 1]
[1 1]

[0 0]
[1 1]

[0 0]
[0 0]



<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 5.</h4>
    <p>
        This result can be compared with the result in <a href="https://www.itp.uni-bremen.de/prichter/download/ThetaConst.pdf" target="_blank">[12]</a> (p. 14, example 5.1)
    </p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    There are 2 odd characteristic among them, their sum is also odd. The corresponding vector -vector of Riemann constants -is denoted by $\mathbf{K}$ and 
    $$
        [\mathbf{K}] = [\mathcal{A}_{2,0}+\mathcal{A}_{4,0}]
    $$
</p>
</div>

In [29]:
# We sum the eChars elements with indices 2*i +1 (because python counts from 0) and take Mod 2
KCh = sum(matrix(eChars[2 * i+1]) for i in range(genus)) % 2

print(KCh)

[1 1]
[0 1]


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">5. Theta function</h1>
<p>

</p>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 3.</h4>
    <p>        
        The Riemann <i>theta function</i> $\theta(v;\tau)$ defined in terms of normalized coordinates $v$ and normalized period matrix $\tau$, canonicaly is given by
        $$
            \theta(\mathbf{v};\tau) = \sum_{n\in\mathbb{Z}^g} e^{i\pi \mathbf{n}^T \tau \mathbf{n} + 2i \pi \mathbf{n}^T \mathbf{v}}.
        $$
        A theta function with characteristic $[\varepsilon ]$ is defined by
        $$
            \theta[\varepsilon](\mathbf{v};\tau) = 
            \theta 
                \begin{bmatrix}
                    \varepsilon'_1 & \ldots & \varepsilon'_g\\
                    \varepsilon_1 & \ldots & \varepsilon_g\\
                \end{bmatrix}
            (\mathbf{v};\tau)=
            \sum_{n\in\mathbb{Z}^g} e^{i\pi\{ (\mathbf{n}+ \frac{1}{2}\mathbf{\varepsilon}'^T ) \tau (\mathbf{n}+ \frac{1}{2}\mathbf{\varepsilon}') +2 (\mathbf{v}+\frac{1}{2}\mathbf{\varepsilon})^T (\mathbf{n}+\frac{1}{2}\mathbf{\varepsilon}') \} }.
        $$
    </p>    
    </div>

</div>

<div style="background-color:  rgba(255, 153, 153, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(255, 51,51); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(255, 51,51);">Important 2.</h4>
    <p>
        In the literature, it is common to use a convention with slightly different characteristics:
        $$
            (\varepsilon_i, \varepsilon'_j) \to 2 (\varepsilon_i, \varepsilon'_j)
        $$
        in consequence
        $$
            \theta[\varepsilon](\mathbf{v};\tau) = 
            \theta 
                \begin{bmatrix}
                    \varepsilon'_1 & \ldots & \varepsilon'_g\\
                    \varepsilon_1 & \ldots & \varepsilon_g\\
                \end{bmatrix}
            (\mathbf{v};\tau)=
            \sum_{n\in\mathbb{Z}^g} e^{i\pi\{ (\mathbf{n}+ \mathbf{\varepsilon}'^T ) \tau (\mathbf{n}+ \mathbf{\varepsilon}') +2 (\mathbf{v}+\mathbf{\varepsilon})^T (\mathbf{n}+\mathbf{\varepsilon}') \} }.
        $$
    </p>    
    </div>

</div>

In [30]:
def Theta( v, ttau, NAcc):
    # NAcc is responsible for the number of elements in the sum, i.e. the precision of the result. 
    # Experimentally, a good approximation is obtained for NAcc>4, but of course this can be increased as needed.
    total_sum = 0
    # epsilon_m is the list [epsilon 1, epsilon 2] where epsilon1 and epsilon2 are vectors
    
    # We iterate over two indices from -NAcc to NAcc
    for n1 in range(-NAcc, NAcc):
        for n2 in range(-NAcc, NAcc):
            # We create vector n
            n = vector([n1, n2])
                    
            # The first component of the sum
            term1 = I * pi * n * (ttau * n)
                    
            # The second component of the sum
            term2 = 2 * I * pi * n * v
                    
            # We add the exp from these components to the total
            total_sum += exp(term1 + term2)
    
    return total_sum

In [31]:
def ThetaCh(epsilon_m, v, ttau, NAcc):
    # NAcc is responsible for the number of elements in the sum, i.e. the precision of the result. 
    # Experimentally, a good approximation is obtained for NAcc>4, but of course this can be increased as needed.
    total_sum = 0
    # epsilon_m is the list [epsilon 1, epsilon 2] where epsilon1 and epsilon2 are vectors
    epsilon1 = epsilon_m[0]
    epsilon2 = epsilon_m[1]
    
    # We iterate over two indices from -NAcc to NAcc
    for n1 in range(-NAcc, NAcc):
        for n2 in range(-NAcc, NAcc):
            # We create vector n
            n = vector([n1, n2])
                    
            # The first component of the sum
            term1 = I * pi * (n + 1/2 * vector(epsilon1)) * (ttau * (n + 1/2 * vector(epsilon1)))
                    
            # The second component of the sum
            term2 = 2 * I * pi * (n + 1/2 * vector(epsilon1)) * (v + 1/2 * vector(epsilon2))
                    
            # We add the exp from these components to the total
            total_sum += exp(term1 + term2)
    
    return total_sum

<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 6.</h4>
    <p>
        In Wolfram Mathematica exists a corresponding function under the name <code style="color:  rgb(81,206,9);font-size: 13px;">SiegelTheta[$\nu_1$,$\nu_2$]($\Omega,s$)</code>. The relation between our variables and those in Mathematica are as follows
        <ul>
            <li> $\Omega=\tau$ </li>
            <li> $s=v$ </li>
            <li> $\nu_1 = \frac{1}{2} epsilon1$ </li>
            <li> $\nu_2 = \frac{1}{2} epsilon2$   </li> 
        </ul>  
        To test this, you can check the following code in Mathematica
    </p>
        <pre style=" color: #f8f8f2; padding: 1px; border-radius: 5px; font-size: 13px;">
        <code style="color: rgb(81,206,9);">
            ```Mathematica         
            SiegelTheta[{{2, 4}, {4, 2}}, IdentityMatrix[2] I, {1, 2}] // N
        </code>
        </pre>
    <p>
        and compare it with below one
    </p>    
        <pre style=" color: #f8f8f2; padding: 10px; border-radius: 5px; font-size: 13px;">
        <code style="color: rgb(81,206,9)">
            ```SageMath           
            # Defining sample data
            epsilon_m = [vector([4, 8]), vector([8, 4])]
            v = vector([1, 2])
            tau = Matrix([[I, 0], [0, I]])
            NAcc = 5
            # Function call
            result = ThetaCh(epsilon_m, v, tau, NAcc)
            print(result)
        </code>
        </pre>
    <p>
        Additionally, one can define the $\theta$ function in Mathematica in a similar way with the following code:
    </p>
        <pre style=" color: #f8f8f2; padding: 10px; border-radius: 5px; font-size: 13px;">
        <code style="color: rgb(81,206,9);">
        ```Mathematica
        ThetaCh[\[Epsilon]m_, v_, \[Tau]_, NAcc_] :=
         Sum[
          Exp[
           I Pi ({Subscript[n, 1], Subscript[n, 2]} + 
            1/2 \[Epsilon]m[[1]]) . (\[Tau] . ({Subscript[n, 1], Subscript[n, 2]} + 1/2 \[Epsilon]m[[1]])) + 2 I Pi ({Subscript[n, 1], Subscript[n, 2]} + 1/2 \[Epsilon]m[[1]]) . (v + 1/2 \[Epsilon]m[[2]])
           ],
          {Subscript[n, 2], -NAcc, NAcc}, {Subscript[n, 1], -NAcc, NAcc}
          ]
        </code>
        </pre>
    <p>
        And check the result of the above test with: 
    </p>
        <pre style=" color: #f8f8f2; padding: 10px; border-radius: 5px; font-size: 13px;">
        <code style="color: rgb(81,206,9);">
        ```Mathematica    
        ThetaCh[2 {{2, 4}, {4, 2}}, {1, 2}, IdentityMatrix[2] I, 5] // N
        </code>
        </pre>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">5.1. Tests</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Test of the formula (<a href="https://arxiv.org/abs/2407.05632" target="_blank">[1]</a>, p.4, eq. 7)
$$
    \theta[\varepsilon](\mathbf{v};\tau) = 
            e^{i\pi\left(\frac{1}{2} \mathbf{\varepsilon}'^T\right) \tau \left(\frac{1}{2} \mathbf{\varepsilon}'\right) + 2i\pi \left( \mathbf{v} + \frac{1}{2} \mathbf{\varepsilon}\right)^T \left(\frac{1}{2} \mathbf{\varepsilon}'\right)} \theta\left(\mathbf{v} + \frac{1}{2}\mathbf{\varepsilon} + \tau\left(\frac{1}{2}\mathbf{\varepsilon}'\right);\tau \right).   
$$    

where a characteristic is a a $2\times g$ matrix $[\varepsilon] = (\mathbf{\varepsilon}',\mathbf{\varepsilon})^T $    
</p>
</div>

In [32]:
# Define the genus
g = 2  # For genus 2

Acc=20

# Define the period matrix tau
### tau = Matrix(CC, [[I, 0.5], [0.5, I]])
tau = omega.inverse() * omegaP

# Define the vector v
v = vector(CC, [0.1, 0.2])

# Define the characteristic
eps_prime = [1, 0]
eps = [0, 1]
N = 2  # Level of the characteristic
char = Matrix([eps_prime,eps])

# Compute theta with characteristic at v
theta_char = ThetaCh(char, v, tau, Acc)

# Compute half of the characteristic vectors
eps_vec = vector(CC, eps)
eps_prime_vec = vector(CC, eps_prime)
eps_half = 0.5 * eps_vec
eps_prime_half = 0.5 * eps_prime_vec

# Compute the shifted vector v_shifted
v_shifted = v + eps_half + tau * eps_prime_half

# Compute the terms for the exponential factor
term1 = (eps_prime_half) * tau * (eps_prime_half)
term2 = (v + eps_half) * (eps_prime_half)

# Compute the exponential factor
exp_factor = exp(I * pi * term1 + 2 * I * pi * term2)

# Compute theta at the shifted point without characteristic
theta_standard = Theta(v_shifted, tau, Acc)


# Compute the RHS of the relation
RHS = exp_factor * theta_standard

# Compute the difference
difference = theta_char - RHS

# Print the results
print("Theta with characteristic:\n", theta_char)
print()
print("Exponential factor:\n", exp_factor)
print()
print("Theta at shifted point:\n", theta_standard)
print()
print("RHS:\n", RHS)
print()
print("Difference:\n", difference)

# Check if the difference is within an acceptable tolerance
tolerance = 1e-12  # Adjust based on the precision
if abs(difference) < tolerance:
    print("The relation is verified within the given tolerance.")
else:
    print("The relation is not satisfied within the given tolerance.")

Theta with characteristic:
 1.0600858517074845 - 0.2501023731212625*I

Exponential factor:
 0.5734712442681018 + 0.050148801540618036*I

Theta at shifted point:
 1.7966652347912195 - 0.5932345951363659*I

RHS:
 1.0600858517074845 - 0.2501023731212626*I

Difference:
 1.1102230246251565e-16*I
The relation is verified within the given tolerance.


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">6. $\sigma$-Functions</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    The theory on which this and the following sections are based comes from the works:
    <a href="https://arxiv.org/pdf/solv-int/9603005" target="_blank">[4]</a>, 
    <a href="https://arxiv.org/pdf/1810.11079" target="_blank">[5]</a>,
    <a href="https://arxiv.org/pdf/1711.08395" target="_blank">[6]</a>,
    <a href="https://arxiv.org/pdf/1106.2408" target="_blank">[7]</a>.
</p>
</div>   

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.</h4>
    <p>        
        <em> Sigma function </em> (Kleinian sigma) is a modular invariant entire function on $\mathrm{Jac}(\mathscr{C})$. It is definef by a relation with the theta function:
        $$
            \sigma(\mathbf{u}) = C \tilde{\sigma}(\mathbf{u})
        $$
        where
        $$
            \tilde{\sigma}(\mathbf{u})= e^{-\frac{1}{2}\mathbf{u}^T \kappa \mathbf{u}} \theta[ K ] (\omega^{-1} \mathbf{u}, \omega^{-1} \omega'),
        $$
        $$
            C= \sqrt{\frac{\pi^g}{\det{\omega}}} \left( \prod_{1\leq i<j \leq 2g+1} (e_i - e_j) \right)^{-1/4},
        $$
        and $[K]$ denotes the characteristics of the vector of Riemann constants. (The expression for $C$ comes from <a href="https://arxiv.org/pdf/1106.2408" target="_blank">[7]</a>, p.9, Eq. II.41)
    </p>    
    </div>

</div>

In [33]:
# We define variables
var('U1 U3')

# We define the accuracy of theta function
Acc=20


# sigma
def Tsigma(U1, U3):
    e = exp(-(1/2)*(vector([U1, U3])*kappa*vector([U1, U3])))
    theta = ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)
    return e*theta

# C constant
det_omega = omega.determinant()
g = 2
#Branch points
BP = find_branch_points(l2, l4, l6, l8, l10)
# Calculating the product of branch point differences
prod = 1
for i in range(len(BP)):
    for j in range(i+1, len(BP)):
        if BP[i] != infinity and BP[j] != infinity:
            prod *= (BP[i] - BP[j])
C = sqrt(pi**g / det_omega) * prod**(-1/4)


def sigma(U1, U3):
    return C*Tsigma(U1, U3)

In [ ]:
# sigma1
def Tsigma1(u1_val, u3_val):
    U1, U3 = var('U1 U3')
    sigma_expr = Tsigma(U1, U3)
    sigma1 =diff (sigma_expr, U1)
    return sigma1.subs({U1: u1_val, U3: u3_val}).n()

# sigma3
def Tsigma3(u1_val, u3_val):
    U1, U3 = var('U1 U3')
    sigma_expr = Tsigma(U1, U3)
    sigma3 =diff (sigma_expr, U3)
    return sigma3.subs({U1: u1_val, U3: u3_val}).n()

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">6.1. Bolza formula</h2>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 5.</h4>
    <p> From <a href="https://arxiv.org/abs/2407.05632" target="_blank">[1]</a>. (Sec. 2.4), we have the Bolza formula:
        $$
            e_\iota = - \frac{\partial_{u_3}\theta[{\iota}](\omega^{-1}u)}{\partial_{u_1}\theta[{\iota}](\omega^{-1}u)}|_{u=0},
        $$
        where $[{\iota}]$ denotes the characteristic corresponding to a branch point $e{\iota}$. It can also be defined in an equivalent way:
        $$
            e_\iota = - \frac{\sigma_3(u_\iota)}{\sigma_1(u_\iota)},
        $$
        where $u_\iota=\frac{1}{2}\omega \epsilon_\iota + \frac{1}{2}\omega' \epsilon'_\iota$.
   <p>
   </div>   

In [183]:
epsp1 = vector(eChars[0][0])
eps1 = vector(eChars[0][1])

epsp2 = vector(eChars[1][0])
eps2 = vector(eChars[1][1])

epsp3 = vector(eChars[2][0])
eps3 = vector(eChars[2][1])

epsp4 = vector(eChars[3][0])
eps4 = vector(eChars[3][1])

epsp5 = vector(eChars[4][0])
eps5 = vector(eChars[4][1])

epsp6 = vector(eChars[5][0])
eps6 = vector(eChars[5][1])

In [184]:
up1 = (1/2)*omega*eps1 + (1/2)*omegaP*epsp1
up2 = (1/2)*omega*eps2 + (1/2)*omegaP*epsp2
up3 = (1/2)*omega*eps3 + (1/2)*omegaP*epsp3
up4 = (1/2)*omega*eps4 + (1/2)*omegaP*epsp4
up5 = (1/2)*omega*eps5 + (1/2)*omegaP*epsp5
up6 = (1/2)*omega*eps6 + (1/2)*omegaP*epsp6

In [185]:
# ei
def e(u1_val, u3_val):
    sigma1 = Tsigma1(u1_val, u3_val)
    sigma3 = Tsigma3(u1_val, u3_val)
    expr = -sigma3/sigma1
    return expr.n()

In [186]:
print("Branch points from Bolza formula:", 
      e(up1[0],up1[1]), 
      e(up2[0],up2[1]), 
      e(up3[0],up3[1]), 
      e(up4[0],up4[1]), 
      e(up5[0],up5[1]), 
      e(up6[0],up6[1]),sep='\n')

Branch points from Bolza formula:
-1.05264146874943 - 5.55111512312578e-16*I
0.695037784917798 + 1.04164546422316*I
-11.8251161409668 - 4.34097202628436e-14*I
2.53231492223134e15 - 6.84742393413673e15*I
0.695037784917799 - 1.04164546422316*I
-0.512317960119317


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Ok, let's see how the $\sigma_i$ functions behave at the points $up_4$</p>
</div>

In [103]:
print("sigma1:", Tsigma1(up4[0],up4[1]))
print("sigma3:", Tsigma3(up4[0],up4[1]))

sigma1: 1.23432198410658e-15 + 3.52430396044750e-15*I
sigma3: -27.2580952691324 - 0.472721613117690*I


<div style="background-color:  rgba(216, 220, 97, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(205,210,44); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(205,210,44);">Observation 1.</h3>
    <p>
     $\sigma_1$=0 so $up4$ is a special divisor
</p>
</d>

In [38]:
print("Branch points:", *branch_points, sep='\n')

Branch points:
-11.8251161409669
-1.05264146874943
-0.512317960119317
0.695037784917798 - 1.04164546422316*I
0.695037784917798 + 1.04164546422316*I
+Infinity


<div style="background-color:  rgba(216, 220, 97, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(205,210,44); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(205,210,44);">Observation 2.</h3>
    <p>
     The order of the elements should be the same. The order of branch points could be wrong because you use the argument $u[\varepsilon]$ instead of $u[\varepsilon+K]$. Based on the discussion <a href="https://community.wolfram.com/groups/-/m/t/3296279" target="_blank">[13]</a> in Wolfram community about the vector of Riemann constants fora a computed characteristic $[\varepsilon]$, before appling the shift by $[K]$ we check wheather 
$$
        \sigma(u[K]) =0 \;\; \text{and} \;\; \sigma_1(u[K]) =0,
$$        
where
$$
        u_\iota=
            \frac{1}{2}\omega 
                \begin{pmatrix}
                    K_{1,1}\\
                    K_{2,1}
                \end{pmatrix}  + 
            \frac{1}{2}\omega' 
                \begin{pmatrix}
                    K_{1,2}\\
                    K_{2,2}
                \end{pmatrix} 
        $$
</p>
</d>

In [107]:
vK1 = vector(KCh[0])
vK2 = vector(KCh[1])
uK = (1/2)*omega*vK1 + (1/2)*omegaP*vK2

print("sigma(u[K])=",Tsigma(uK[0], uK[1]))
print("sigma1(u[K])=",Tsigma1(uK[0], uK[1]))

sigma(u[K])= 4.168869563487747e-16 + 4.900022606845388e-16*I
sigma1(u[K])= 1.23432198410658e-15 + 3.52430396044750e-15*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Now let's check $u$ vectors with shifted characteristic:
$$
    [\tilde{\varepsilon_\iota}]=[\varepsilon_\iota + K] = \left( [\varepsilon_\iota] + [K] \right)\pmod{2}
$$    
$$
    \tilde{u}_\iota=\frac{1}{2}\omega \tilde{\varepsilon_\iota} + \frac{1}{2} \omega'  \tilde{\varepsilon_\iota}'
$$    
</p>
</div>

In [146]:
#up shifted
sChar1 = (matrix(eChars[0]) + KCh)%2
sepsp1 = vector(sChar1[0])
seps1 = vector(sChar1[1])

sChar2 = (matrix(eChars[1]) + KCh)%2
sepsp2 = vector(sChar2[0])
seps2 = vector(sChar2[1])

sChar3 = (matrix(eChars[2]) + KCh)%2
sepsp3 = vector(sChar3[0])
seps3 = vector(sChar3[1])

sChar4 = (matrix(eChars[3]) + KCh)%2
sepsp4 = vector(sChar4[0])
seps4 = vector(sChar4[1])

sChar5 = (matrix(eChars[4]) + KCh)%2
sepsp5 = vector(sChar5[0])
seps5 = vector(sChar5[1])

sChar6 = (matrix(eChars[5]) + KCh)%2
sepsp6 = vector(sChar5[0])
seps6 = vector(sChar5[1])

ups1 = (1/2)*omega*seps1 + (1/2)*omegaP*sepsp1
ups2 = (1/2)*omega*seps2 + (1/2)*omegaP*sepsp2
ups3 = (1/2)*omega*seps3 + (1/2)*omegaP*sepsp3
ups4 = (1/2)*omega*seps4 + (1/2)*omegaP*sepsp4
ups5 = (1/2)*omega*seps5 + (1/2)*omegaP*sepsp5
ups6 = (1/2)*omega*seps6 + (1/2)*omegaP*sepsp6

print("Branch points from Bolza formula with u[epsilon+K]:", 
      e(ups1[0],ups1[1]), 
      e(ups2[0],ups2[1]), 
      e(ups3[0],ups3[1]), 
      e(ups4[0],ups4[1]), 
      e(ups5[0],ups5[1]), 
      e(ups6[0],ups6[1]),sep='\n')

Branch points from Bolza formula with u[epsilon+K]:
-0.431121078698602 + 0.424863540601260*I
2.53231492223134e15 - 6.84742393413673e15*I
-33.5168254120767 + 12.1374912809649*I
0.695037784917798 + 1.04164546422316*I
0.258106930440267 + 2.91433543964104e-16*I
0.258106930440267 + 2.91433543964104e-16*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">7. $\wp$-Functions</h1>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 6.</h4>
    <p>        
        Multiply periodic Klein-Weierstrass $\wp$-functions are defined by
        $$
            \wp_{ij}(\mathbf{u}):= - \frac{\partial^2\log{\sigma(\mathbf{u})}}{\partial u_i \partial u_j}, \quad \wp_{ijk}:=- \frac{\partial^3\log{\sigma(\mathbf{u})}}{\partial u_i \partial u_j \partial u_k},
        $$
        where $\sigma(u)$ is called <i>sigma function</i>. This can be also written by 
        $$
            \wp_{ij} = \frac{\sigma_i(\mathbf{u})\sigma_j(\mathbf{u}) - \sigma(\mathbf{u})\sigma_{ij}(\mathbf{u})}{\sigma^2(\mathbf{u})} = \frac{\tilde{\sigma}_i(\mathbf{u})\tilde{\sigma}_j(\mathbf{u}) - \tilde{\sigma}(\mathbf{u})\tilde{\sigma}_{ij}(\mathbf{u})}{\tilde{\sigma}^2(\mathbf{u})},
        $$
        where $\sigma_i(\mathbf{u})$ denotes the derivative of the sigma function with respect to the $i$-th component of $\mathbf{u}$. <br>
        <br>
        It can be shown that the above definitions can be written in the form
        $$
            \wp_{ij}:=\kappa_{ij} - \frac{\partial^2}{\partial u_i \partial u_j}\log{\theta[K](\omega^{-1}u;\tau)}, \quad \wp_{ijk}:=- \frac{\partial^3}{\partial u_i \partial u_j \partial u_k}\log{\theta[K](\omega^{-1}u;\tau)}.
        $$
    </p>    
    </div>

</div>

In [268]:
#definition with thetas
# We define variables
var('U1 U3')

# We define the accuracy of theta function
Acc=20


# WeierstrassP11
def WeierstrassP11(u1_val, u3_val):
    symbolic_expr = kappa[0, 0] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U1, 2)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP13
def WeierstrassP13(u1_val, u3_val):
    symbolic_expr = kappa[0, 1] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)),  U1, U3)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP33
def WeierstrassP33(u1_val, u3_val):
    symbolic_expr = kappa[1, 1] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U3, 2)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP1111
def WeierstrassP1111(u1_val, u3_val):
    symbolic_expr = - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U1, 4)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP3333
def WeierstrassP3333(u1_val, u3_val):
    symbolic_expr = - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U3, 4)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">7.1. Tests</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    As a first test of these functions, we can check if they satisfy the key property  
$$
    \wp_{ij}(\mathbf{u}+2\omega \mathbf{n} + 2\omega'\mathbf{n}') = \wp_{ij}(\mathbf{u}),
$$
where
$$
    \mathbf{n} = 
        \begin{pmatrix}
            n_1\\
            n_2
        \end{pmatrix}, \quad
    \mathbf{n}' = 
        \begin{pmatrix}
            n'_1\\
            n'_2
        \end{pmatrix} \in \mathbb{Z}^2.
$$  
</p>
</d>

In [40]:
ntest = vector([1, 2])
nPtest = vector([-3, -5])

wn = omega*ntest
wPn= omegaP*nPtest

print("Theta based definitions")
print()
print("Test P11:")
print(WeierstrassP11(2.0, 3.0) - WeierstrassP11(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P13:")
print(WeierstrassP13(2.0, 3.0) - WeierstrassP13(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P33:")
print(WeierstrassP33(2.0, 3.0) - WeierstrassP33(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print()
print("Sigma based definitions")
print()
print("Test P11:")
print(Weierstrass2P11(2.0, 3.0) - Weierstrass2P11(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P13:")
print(Weierstrass2P13(2.0, 3.0) - Weierstrass2P13(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P33:")
print(Weierstrass2P33(2.0, 3.0) - Weierstrass2P33(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))

Theta based definitions

Test P11:
7.36690708436072e-11 + 4.91695573145989e-12*I

Test P13:
-1.74793512996985e-11 - 3.78008735424373e-11*I

Test P33:
1.20792265079217e-11 - 1.58024704433046e-11*I


Sigma based definitions

Test P11:
-3.83515441626514e-11 - 9.03901593783867e-13*I

Test P13:
2.50466314355435e-13 - 4.64994659973170e-13*I

Test P33:
3.62376795237651e-12 + 1.52677602519117e-11*I


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">7.2. Basic relations</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h3 style="font-size: 24px;">I. </h3>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 7</h4>
    <p>  
        
The following formula cames from <a href="https://arxiv.org/pdf/2312.10859" target="_blank">[11]</a> (Eq. 44) - this is KdV equation

$$
       \wp_{1111} = 6\wp^2_{11} +  4\lambda_2 \wp_{11} + 4 \wp_{13} + 2 \lambda_4 
    $$

</pre>
</div>

In [314]:
u1 = 0.212131
u3 = -2.231

RHS = 6 * WeierstrassP11(u1,u3)^2 + 4 * l2 * WeierstrassP11(u1,u3) + 4 * WeierstrassP13(u1,u3) + 2*l4
LHS = WeierstrassP1111(u1,u3)

LHS - RHS

-1.81898940354586e-12 - 2.72901563345651e-12*I

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
Let's check this for another value of vector $u$

</p>
</div>

In [315]:
u1 = -1.98213
u3 = 3.76213

RHS = 6 * WeierstrassP11(u1,u3)^2 + 4 * l2 * WeierstrassP11(u1,u3) + 4 * WeierstrassP13(u1,u3) + 2*l4
LHS = WeierstrassP1111(u1,u3)

LHS - RHS

9.86233317235019e-12 + 9.49285094975494e-12*I

<div style="background-color:  rgba(216, 220, 97, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(205,210,44); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(205,210,44);">Observation 3.</h3>
    <p>
    Works great :)
</p>
</d>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h3 style="font-size: 24px;">II. </h3>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 8</h4>
    <p>  
        
Based on <a href="https://arxiv.org/pdf/1106.2408" target="_blank">[7]</a></li> (5.10-13) let's take
$$
    \Omega_{ij} = \frac{1}{2} \omega (\epsilon_i + \epsilon_j) +  \frac{1}{2} \omega' (\epsilon'_i + \epsilon'_j),
$$
then
$$
    \wp_{11}(\Omega_{ij}) = e_i + e_j, \quad \wp_{13}(\Omega_{ij}) = -e_i e_j,  \quad i,j=1,\ldots,5, \; i\neq j
$$
</pre>
</div>

In [313]:
Omega12 = (1/2)*omega*(eps1 + eps2) + (1/2)*omegaP*(epsp1 + epsp2)
Omega13 = (1/2)*omega*(eps1 + eps3) + (1/2)*omegaP*(epsp1 + epsp3)
Omega15 = (1/2)*omega*(eps1 + eps5) + (1/2)*omegaP*(epsp1 + epsp5)
Omega23 = (1/2)*omega*(eps2 + eps3) + (1/2)*omegaP*(epsp2 + epsp3)
Omega25 = (1/2)*omega*(eps2 + eps5) + (1/2)*omegaP*(epsp2 + epsp5)
Omega35 = (1/2)*omega*(eps3 + eps5) + (1/2)*omegaP*(epsp3 + epsp5)

In [405]:
print("P11(Omega12)=",WeierstrassP11(Omega12[0],Omega12[1]))
print("From theory: e1+e2=",e(up1[0],up1[1])+ e(up2[0],up2[1]))
print("From practice: e3+e5=",e(up3[0],up3[1])+ e(up5[0],up5[1]))

P11(Omega12)= -11.1300783560491 - 1.04164546422316*I
From theory: e1+e2= -0.357603683831631 + 1.04164546422316*I
From practice: e3+e5= -11.1300783560490 - 1.04164546422321*I


In [406]:
print("P13(Omega12)=",WeierstrassP13(Omega12[0],Omega12[1]))
print("From theory: -e1*e2=",-e(up1[0],up1[1])* e(up2[0],up2[1]))
print("From practice: -e3*e5=",-e(up3[0],up3[1]) * e(up5[0],up5[1]))

P13(Omega12)= 8.21890252901329 - 12.3175785921502*I
From theory: -e1*e2= 0.731625594752220 + 1.09647921137605*I
From practice: -e3*e5= 8.21890252901333 - 12.3175785921502*I


In [407]:
print("P11(Omega13)=",WeierstrassP11(Omega13[0],Omega13[1]))
print("From theory: e1+e3=",e(up1[0],up1[1])+ e(up3[0],up3[1]))
print("From practice: e2+e5=",e(up5[0],up5[1]) + e(up2[0],up2[1]))

P11(Omega13)= 1.39007556983559 + 9.15933995315754e-15*I
From theory: e1+e3= -12.8777576097163 - 4.37991975292688e-14*I
From practice: e2+e5= 1.39007556983560 - 6.66133814775094e-16*I


In [408]:
print("P11(Omega15)=",WeierstrassP11(Omega15[0],Omega15[1]))
print("From theory: e1+e5=",e(up1[0],up1[1])+ e(up5[0],up5[1]))
print("From practice: e2+e3=",e(up3[0],up3[1]) + e(up2[0],up2[1]))

P11(Omega15)= -11.1300783560490 + 1.04164546422316*I
From theory: e1+e5= -0.357603683831630 - 1.04164546422316*I
From practice: e2+e3= -11.1300783560490 + 1.04164546422312*I


In [298]:
print("Branch points from Bolza formula:", 
      e(up1[0],up1[1]), 
      e(up2[0],up2[1]), 
      e(up3[0],up3[1]), 
      e(up4[0],up4[1]), 
      e(up5[0],up5[1]), sep='\n')

Branch points from Bolza formula:
-1.05264146874943 - 4.34176838933187e-16*I
0.695037784917798 + 1.04164546422316*I
-11.8251161409668 - 4.33650206903357e-14*I
2.53231492223134e15 - 6.84742393413673e15*I
0.695037784917799 - 1.04164546422316*I


In [299]:
print("Branch points:", *branch_points, sep='\n')

Branch points:
-11.8251161409669
-1.05264146874943
-0.512317960119317
0.695037784917798 - 1.04164546422316*I
0.695037784917798 + 1.04164546422316*I
+Infinity


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 24px;">8. Jacobi inversion problem on branch points</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    Let $\mathbf{u}=\mathcal{A}(D)$ be the Abel image of a degree g positive non-special divisor $D$ on the curve. Then $D$ is uniquely defined by the system of equations:
    $$
        \mathcal{R}_{2g}(x;\mathbf{u}) = x^g - \sum_{i=1}^g x^{g-1} \wp_{1,2i-1}(\mathbf{u}) =0,
    $$
    $$
        \mathcal{R}_{2g+1}(x,y;\mathbf{u}) = 2y + \sum_{i=1}^{g-i} x^{g-1} \wp_{1,1,2i-1}(\mathbf{u}) =0.
    $$
    So in case of $g=2$:    
    $$
        \mathcal{R}_4(x;\mathbf{u})=x^2 - x\wp_{11}(\mathbf{u})- \wp_{13}(\mathbf{u})=0,
    $$    
    $$
        \mathcal{R}_5 (x;\mathbf{u})= 2y + x\wp_{111}(\mathbf{u}) + \wp_{113}(\mathbf{u})=0.
    $$    
</p>
</d>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">8.1. Divisor</h2>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 9.</h3>
    <p>        
        Let 
        $$
            D=\sum_{i=1}^n P_i,
        $$
        be a divisor on a curve $\mathscr{C}$. We assume that $D$ is <b>non-special</b>, that is 
        <ul>
          <li> $ n\geq g$,
          <li> $D$ does not contain pairs of points connected by the hyperelliptic involution.
        </ul>
    </p>    
</div>

<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(0, 128,255);">Note 8.</h3>
    <p>
    On special divisors $\sigma$-function vanishes, and so $\wp$-functions have singularities.
  </p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
As a point $P_i$ we choose
$$
    P_i = (x_i,y_i) = (x_i, y_{+}(x_i)),
$$
where $y_{+}(x) = + \sqrt{y^2(x)}$ (there is second option $y_{-}(x) = - \sqrt{y^2(x)}$ ). 

In our case let's take
$$
    P_1 = (x_1, y_1)=(1.5,y_{+}(1.5))
$$
$$
    P_2 = (x_2, y_2)=(3.1,y_{+}(3.1))
$$
</p> 
</div>

In [368]:
# Definition of y function 
def y(x):
    res = sqrt(x^5 + l2*x^4 + l4*x^3 + l6*x^2 + l8*x + l10)
    return res.n()

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    To calculate $\mathbf{u}$ I will use the Sage function:  <code style="color: #66d9ef;font-size: 13px;">abel_jacobi()</code>, 
    where it expects lists of tuples in the format  <code style="color: #66d9ef;font-size: 13px;font-size: 13px;">(v,P)</code>, where  <code style="color: #66d9ef;font-size: 13px;">v</code> is the multiplicity of the point in the divisor (in our case 1 for both points), and  <code style="color: #66d9ef;font-size: 13px;">P</code> is a tuple  <code style="color: #66d9ef;font-size: 13px;">(x,y)</code> representing the point on the curve. 

The multiplicity <code style="color: #66d9ef;font-size: 13px;">v</code> (also called valuation) in the context of divisors determines how many times a given point appears in the divisor. Here are some key points:
<ul>
    <li> For regular points on the curve that are neither singular points nor points at infinity, typically $v = 1$ or $v = -1$ is used.
$v = 1$ means the point appears positively in the divisor (it is "added").
    <li> $v = -1$ means the point appears negatively in the divisor (it is "subtracted").
    <li> If a point appears multiple times, $v$ can be greater than 1 or less than -1.
    <li> For special points (e.g., points at infinity or singular points), v may take other values depending on the local structure of the curve at that point.
</ul>

In your case, where you define the divisor as a sum of two regular points: <code style="color: #66d9ef;font-size: 13px;">D = [x1 + x2, y1 + y2]</code>

each of these points appears once positively in the divisor, so for both points <code style="color: #66d9ef;font-size: 13px;">v = 1</code>.
Therefore, divisor in the format suitable for the <code style="color: #66d9ef;font-size: 13px;">abel_jacobi()</code> function would look like this:    
</p>
</div> 

In [276]:
x1 = 0.256
x2 = 11.721

y1 = y(x1)
y2 = y(x2)

# P_i points
P1 = [ x1, y1 ] 
P2 = [ x2, y2 ]

# Divisor
divisor = [(1, (x1, y1)), (1, (x2, y2))]

<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(0, 128,255);">Note 9.</h3>
    <p>
    If one wanted to define a more complex divisor, for example, the difference of two points, it might look like this:<br>
        <code style="color: #66d9ef;font-size: 13px;">divisor = [(1, (x1, y1)), (-1, (x2, y2))]</code><br>
    where -1 indicates that the second point is subtracted in the divisor.
  </p>
</div>

In [277]:
AJ = S.abel_jacobi(divisor)

print(AJ)

(-0.095227328487916328568064649224 - 0.47103938123276306779408678278*I, -0.94610300133296732955826058561 - 0.36651365565790567383047992719*I)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">8.2. Tests</h2>
</div>

In [278]:
u1=AJ[0]
u3=AJ[1]

print("P11(u):")
print(WeierstrassP11(u1, u3))

print("P13(u):")
print(WeierstrassP13(u1, u3))

print("P33(u):")
print(WeierstrassP33(u1, u3))

P11(u):
6.36829653384855 - 4.59081552648907*I
P13(u):
-9.85938695314929 - 1.97507933943479*I
P33(u):
-23.8259011047188 + 49.1062328287844*I


In [279]:
WeierstrassP11(u1, u3).abs()

7.85052787660044

In [280]:
u1=AJ[0]
u3=AJ[1]

print("P11(u):")
print(WeierstrassP11(u3, u1))

print("P13(u):")
print(WeierstrassP13(u3, u1))

print("P33(u):")
print(WeierstrassP33(u3, u1))

P11(u):
-3.56865567641634 + 5.00008457846782*I
P13(u):
-3.58050573425828 + 5.20008622575496*I
P33(u):
13.3973281479906 + 1.55914156214922*I


In [281]:
mat1R4=[[1, x, x^2],[ 1, x1, x1^2], [1, x2, x2^2]]
det1=matrix(mat1R4).determinant()
mat2R4=[[ 1, x1], [1, x2]]
det2=matrix(mat2R4).determinant()
det1/det2

x^2 - 11.9770000000000*x + 3.00057600000000

In [282]:
(x1*x2)/(x1+x2)

0.250528179009769

In [283]:
(x1+x2)/(x1*x2)

3.99156695247846

In [284]:
WeierstrassP13(u3, u1)/WeierstrassP11(u3, u1)

1.02762109946350 - 0.0173437337281857*I

In [285]:
WeierstrassP11(u3, u1)/WeierstrassP13(u3, u1)

0.972844202700385 + 0.0164192335282465*I

In [286]:
WeierstrassP13(u1, u3)/WeierstrassP11(u1, u3)

-0.871646731391295 - 0.938500375375504*I

In [287]:
WeierstrassP11(u1, u3)/WeierstrassP13(u1, u3)

-0.531313407866428 + 0.572064134203511*I

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
We should get:
$$
        x^2 - \wp_{11}(\mathbf{u}) \;x - \wp_{13}(\mathbf{u})=0,
$$
</p>
</div> 

<div style="background-color:  rgba(216, 220, 97, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(205,210,44); padding: 15px; border-radius: 5px; ">
    <h3 style="font-size: 20px;color: rgb(205,210,44);">Observation 4.</h3>
    <p>
    Ok, this is bad.
  </p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;"> Literature </h1>
    <ol type="1">
        <li> 
            Julia Bernatska, <em>Computation of $\wp$-functions on plane algebraic curves</em>,  arXiv (2024),<br>
            URL: <a href="https://arxiv.org/abs/2407.05632" target="_blank">https://arxiv.org/abs/2407.05632</a> </li>
        <li>
            Julia Bernatska, <em>Uniformization of a genus 4 hyperelliptic curve with arbitrary complex branch points</em>, Wolfram Community (2024),<br>
            URL: <a href="https://community.wolfram.com/groups/-/m/t/3243472" target="_blank">https://community.wolfram.com/groups/-/m/t/3243472</a> </li>
        <li> 
            SageMath Documentation, <em>Riemann matrices and endomorphism rings of algebraic Riemann surfaces</em>,<br>
            URL:<a href="https://doc.sagemath.org/html/en/reference/curves/sage/schemes/riemann_surfaces/riemann_surface.html#sage.schemes.riemann_surfaces.riemann_surface.RiemannSurface"
target="_blank">https://doc.sagemath.org/html/en/reference/curves/sage/schemes/riemann_surfaces/riemann_surface.html#sage.schemes.riemann_surfaces.riemann_surface.RiemannSurface</a></li>
        <li> 
            V.M.Buchstaber, V.Z. Enolski, and D.V.Leykin, <em>Hyperelliptic Kleinian function and applications</em>, arXiv (1996), <br>
            URL: <a href="https://arxiv.org/pdf/solv-int/9603005" target="_blank">https://arxiv.org/pdf/solv-int/9603005</a></li>
        <li> 
            V.M.Buchstaber, V.Z. Enolski, and D.V.Leykin, <em>Multi-variable sigma-functions: old and new results</em>, arXiv (2018), <br>
            URL: <a href="https://arxiv.org/pdf/1810.11079" target="_blank">https://arxiv.org/pdf/1810.11079</a></li>
        <li> 
            J. C. Eilbeck, J. Gibbons, Y. Onishi, and S. Yasuda, <em>Theory of Heat Equations for Sigma Functions</em>, arXiv (2018), <br>
            URL: <a href="https://arxiv.org/pdf/1711.08395" target="_blank">https://arxiv.org/pdf/1711.08395</a></li> 
        <li> 
            V. Z. Enolski, B. Hartmann, V. Kagramanova, J. Kunz, C. Lammerzahl, P. Sirimachan, <em>Inversion of a general hyperelliptic integral and particle motion in Horava-Lifshitz black hole space-times </em>, arXiv (2011), <br>
            URL: <a href="https://arxiv.org/pdf/1106.2408" target="_blank">https://arxiv.org/pdf/1106.2408</a></li>
        <li>
            Athorne C., Eilbeck J.C., Enolskii V.Z., <em>A SL(2) covariant theory of genus 2 hyperelliptic functions</em>, Math. Proc. Cambridge Philos. Soc. 136 (2004), 269–286,<br>
            URL: <a href="https://www.cambridge.org/core/journals/mathematical-proceedings-of-the-cambridge-philosophical-society/article/abs/sl2-covariant-theory-of-genus-2-hyperelliptic-functions/2EE97A3BFFF5F70DE28F31560F0AC482" target="_blank"><em>https://www.cambridge.org/core/journals/mathematical-proceedings-of-the-cambridge-philosophical-society/article/abs/sl2-covariant-theory-of-genus-2-hyperelliptic-functions/2EE97A3BFFF5F70DE28F31560F0AC482</a>
        </li>
        <li>
            H. F. Baker, <em>An Introduction to the theory of multiply periodic functions</em>,(1907),<br>
        </li>
        <li>
            M. Hayashi, K. Shigemoto and T. Tsukioka, <em>The half-period addition formulae for genus two hyperelliptic ℘ functions and the Sp(4, R) Lie group structure</em>, J. Phys. Commun. 6 085004 (2022)<br>
            URL: <a href="https://iopscience.iop.org/article/10.1088/2399-6528/ac8521" target="_blank"><em>https://iopscience.iop.org/article/10.1088/2399-6528/ac8521</a>
        </li>
        <li>
            J. Bernatska, <em>Reality conditions for the KdV equation and exact quasi-periodic solutions in finite phase spaces</em>, Journal of Geometry and Physics, 206, 105322 (2024),<br>
            URL: <a href="https://arxiv.org/pdf/2312.10859" target="_blank"><em>https://arxiv.org/pdf/2312.10859</a>
        </li>
        <li>
            Enolski V.Z., Richter P.H.<em>Periods of hyperelliptic integrals expressed in terms of $\theta$-constants by means of Thomae formulae</em>, (2007),<br>
            URL: <a href="https://www.itp.uni-bremen.de/prichter/download/ThetaConst.pdf" target="_blank"><em>https://www.itp.uni-bremen.de/prichter/download/ThetaConst.pdf</a></li>
        <li>
            Julia Bernatska, <em>Vector of Riemann constants</em>, Wolfram Community (2024),<br>
            URL: <a href="https://community.wolfram.com/groups/-/m/t/3296279" target="_blank">https://community.wolfram.com/groups/-/m/t/3296279</a> </li>

</ol>
</div>